# Hist 3368: Week 5 - Interpret word change over time

In this notebook, we'll use your newfound skills at navigating through tabular data to compare the *context* of a single word in its usage at two points of time.

We'll be working with a new concept -- ***collocation***.  Collocation is a reference to words that are located with other words: they are literally co-located.  

You have already seen the code for collocation, but you haven't used that code to rigorously compare the context of the same word at two points in time. This notebook will take you through that process and ask you some discussion questions at the end.

First, we'll investigate the word 'malpractice.' 

* First, we'll inspect the contexts in which the word was used in 1967 and 2010.  We'll call up words that were used alongside 'malpractice' in the same speech, and we'll count them.

* Next we'll qualitatively inspect individual speeches where the word appeared the most -- using our skills at critical reading of primary sources to get more information.  

### Set up packages and data

In [67]:
import nltk
import pandas as pd
import csv
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
stopwords_regex = r'\b(?:{})\b'.format('|'.join(stop))

[nltk_data] Downloading package stopwords to
[nltk_data]     /users/jguldi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
cd /scratch/group/history/hist_3368-jguldi

/scratch/group/history/hist_3368-jguldi


In [69]:
word1 = "malpractice"

In [70]:
# load congress data
congress = pd.read_csv("congress1967-2010.csv")

# narrow to just particular years
congress1967 = congress[congress['year'] == 1967].copy().reset_index()
congress1967 = congress1967.drop(['index', 'Unnamed: 0'], 1) #minor reformatting - drop extra columns

congress2010 = congress[congress['year'] == 2010].copy().reset_index()
congress2010 = congress2010.drop(['index', 'Unnamed: 0'], 1) #minor reformatting - drop extra columns

word1_1967_context = congress1967.copy()
word1_2010_context = congress2010.copy()

### Look for the keyword, word1

We learned that we can use **str.count()** to count the individual occurrences of a word per speech. str.count() takes one object, the word you're looking for. We're looking for *word1*, which is a variable containing the word 'malpractice.'

We can also use **.sort_values()** to tell pandas to sort the data by the values in one column.  .sort_values() takes the arguments "by" (where you tell it which column to use, using quotation marks, and "ascending," which can be True or False, depending on how you want your values arranged.

In [109]:
congress1967['keywordcount'] = congress1967['speech'].str.count(word1)
congress1967.sort_values(by = 'keywordcount', ascending = False)[:5]

,Unnamed: 0.1,speech,date,speaker,word_count,year,month,month_year,keywordcount
44440,44440,Mr. Speaker. Rear Adm. John Harilee. U.S. Navy...,1967-04-20,Mr. DE LA GARZA,3231,1967,4,1967-04-01,2
93562,93562,"Mr. Speaker. the explosive literary work-""Comm...",1967-07-24,Mr. RARICK,7807,1967,7,1967-07-01,1
180701,180701,"says cochairman Russell. ""is an effort by priv...",1967-12-12,The ICBO,1154,1967,12,1967-12-01,1
75197,75197,Mr. Speaker. a number of remarki and insertion...,1967-06-16,Mr. ASHBROOK,281,1967,6,1967-06-01,1
127886,127886,Mr. Speaker. Betty Furness. President Johnsons...,1967-09-25,Mr. PATMAN,146,1967,9,1967-09-01,1


How many times was the word 'malpractice' used in each time period?

In [110]:
congress1967['speech'].str.count(word1).sum()

11

In [111]:
congress2010['speech'].str.count(word1).sum()

92

'Malpractice' is a word that appeared far more frequently in 2010 than in 1967.  

What did it mean in each year? Why did its usage later explode?

We can answer these questions by looking at the *context* in which the word appeared.

### Get the context for word1

To get the context for word1, you must call up the full text for the speeches where the keyword appears.

Next, we can count the words that commonly appear around word1.

Using .str.count(word1) and .nlargest(n) give you a list of the rownumbers for the n speeches where word1 appears the most frequently.  The rownumbers are stored in the .index for the resulting series. We save the results as the variables *word1_speechnumbers*

We can use .loc() and the rownumbers to call up the full text of the speeches that correspond to word1_speechnumbers.  We will save the results as *word1_context.*

We then clean the data using stopwording, lowercasing, and punctuation stripping.

We then count the words that result using commands for breaking the string into words, giving each word a separate row, dropping empty rows, and counting the results:

    str.split().explode().dropna().value_counts()
    
    

In [85]:
# get the rownumbers of the n speeches that use word1 the most
word1_1967_speechnumbers = congress1967['speech'].str.count(word1).nlargest(10) 
word1_2010_speechnumbers = congress2010['speech'].str.count(word1).nlargest(10) 

# get the speeches that mentioned word1 the most
word1_1967_context = congress1967.loc[list(word1_1967_speechnumbers.index)]
word1_2010_context = congress2010.loc[list(word1_2010_speechnumbers.index)]
                                
# clean up the data
word1_1967_context['cleanspeech'] = word1_1967_context['speech'].str.replace('[^\w\s]','').str.lower() # remove punctuation, lowercase
word1_1967_context['stopworded'] = word1_1967_context['cleanspeech'].str.replace(stopwords_regex, '') #stopwording

word1_2010_context['cleanspeech'] = word1_2010_context['speech'].str.replace('[^\w\s]','').str.lower() # remove punctuation, lowercase
word1_2010_context['stopworded'] = word1_2010_context['cleanspeech'].str.replace(stopwords_regex, '') #stopwording

In [86]:
# count the other words that appear in the context of word1 in 1967
word1_1967_context_count = word1_1967_context["stopworded"].str.split().explode().dropna().value_counts()
word1_1967_context_count[:10]

angeles         65
los             65
police          63
watts           57
would           44
communist       41
organization    37
communists      36
one             33
american        33
Name: stopworded, dtype: int64

In [87]:
# count the other words that appear in the context of word1 in 1967
word1_2010_context_count = word1_2010_context["stopworded"].str.split().explode().dropna().value_counts()
word1_2010_context_count[:10]

health         81
would          81
insurance      60
medical        43
people         39
malpractice    39
amendment      37
care           36
bill           36
reform         35
Name: stopworded, dtype: int64

Interestingly, in 1967, the word 'malptractice' doesn't seem to refer to doctors or medicine. It seems to be invoked alongside the Watts Riots, when African Americans rioted for their civil rights, and alongside disucssions of Communism.  

By 2010, the word 'malpractice' is used in a more familiar context -- the context of *medical* malpractice, where a doctor doesn't perform the correct procedure.  It is invoked alongside discussions of 'insurance' and 'health,' and also alongside 'reform.'  

We could stop now, and simply cite these differences as evidence of change over time.  But a stronger approach involves actually *reading* how these words were used in context.

### Inspect the data for the word 'malpractice' in 1967

We can call up *word1_1967_speechnumbers* and find out how frequently 'malpractice' was mentioned in 1967.

In [88]:
word1_1967_speechnumbers

44440     2
40166     1
75197     1
93562     1
120005    1
127886    1
143585    1
175915    1
177228    1
180701    1
Name: speech, dtype: int64

That's not a lot -- we've seen speeches that mention democracy 25 times in a single speech.

Look at the speech that mentions malpractice about -- the speech at row number 44440.

Who gave the speech?

In [89]:
word1_1967_context['speaker'][44440][:1000]

'Mr. DE LA GARZA'

What did the speech say?

In [95]:
word1_1967_context['speech'][44440][:1000]

'Mr. Speaker. Rear Adm. John Harilee. U.S. Navy. retired. is Chairman of the Federal Maritime Commissionand he does a good job. He gets around to see the people who are concerned with the operation of the Maritime Commission. Yesterday. he was down in the 15th Congressionad District of Texas. which I am privileged to represent. Admiral Harllee addressed the Propeller Club in Brownsville. Tex. Since Admiral Harllee made such a splendid speech. I thought the membership might be interested in reading what he had to say. The speech follows: Gentlemen. President Al Roser. I am happy to be back in Texas where my heart is. This is the first time we have been in Brownsville. however. we have heard many things about your city from my good friend and your great Congressman. Kika de la Garza. who invited us to visit your port. I want to say that Congressman de la Garza is doing an outstanding job for you in Washington and sends you his best regards. I have also heard much about your growing port 

I've cut the quote short.  You can see the whole speech for yourself by deleting the [:1000] at the end of the line of code.

Here are the sentences that use the word 'malpractice.'

"We regulate conferences by insuring that they carry out such functions as the following: (1) Selfpolicing. Rebates and other malpractices resulting in unfair competition occur more abroad than in the United States. Some foreign nations take a different view of these practices than we do. but the major maritime nations are gradually beginning to share our views as to the desirability of fair competition. These malpractices can be much more effectively dealt with by selfpolicing bodies than by a Government body alone. although we do the best we can in this regard. 

In context, the 1967 speech invokes 'malpractice' in the context of maritime insurance fraud.

That's very different than the current usage of the word 'malpractice.' Today, the word mainly refers to *medical* malpractice.  Apparently, the word's meaning has changed since 1967.

#### The importance of keeping going

We still don't have a good explanation for why 'malpractice' seems to appear alongside discussions of Communism and the Watts Riots, but we can keep using this same process to inspect each of the top speeches until we reach understanding.

Keep going until you understand! You'll want to quote the evidence you gather.

Here's a speech that reference Los Angeles.

In [96]:
word1_1967_context['speech'][180701]

'says cochairman Russell. "is an effort by private industry to cope with this critical problem and to encourage change of the unnatural pattern of white economic dominance in the Negro community." Cochairman Rodman C. Rockefeller. a vicepresident of the International Basic Economy Corporation. points out that ICBO also "enables white businessmen to do something in addition to just contributing financially to the civil rights movement." IBM. for example. loaned one of its executives. Carter F. Henderson. to ICBO for seven months. paying his salary all the while he worked to set up ICBO as a going operation. PepsiCola loaned. on similar terms. its manager of special markets. Charles Dryden. for two months when ICBO suddenly became shortstaffed. "Many businessmen." Rockefeller added. "have been sitting on their hands. wondering how to participate. Now they are pitching in by doing what comes naturallypromoting good business." But promoting good business is not always easy. especially when

Use command-F or EDIT > FIND in the menu above to search for the word 'malpractice' in your screen until you see the context.  

How was this congressperson talking about malpractice?

Here's a short quote:

"Norman Houston said that he hoped that the Council eventually could find time to work toward eliminating overcharging. extremely high credit terms and other business malpractices that have infuriated the citizens of Watts."

African Americans in Watts, Los Angeles were rioting because they had been discriminated against by banks. This crisis is now well documented. There was literally no way for most African Americans to own a house, pay a mortgage, or take out normal business loans, as a result of racism.  

What we learn from this passage is that Congresspeople referred to such practices as 'business malpractice.'  

The kind of malpractice that concerned Congress in 1967 was racist financial practice, not medical malpractice. 

***Why this is important -- and what we don't know yet***

You have made a historical discovery -- not enough for an entire paper, but enough to have a good research question.  

Follow-up questions that we will not discuss today include these: WHEN did 'malpractice' shift its usage in Congress?  Were there any other shifts besides this one?  WHY did the usage of 'malpractice' shift?  Did Congress stop caring about racist financial malpractice? Or did it talk about the same problem of race and credit using different language? 

In a research paper, you would want to make room to investigate each of these questions in turn until you reached a full and compelling portrait of the shifts in question.

### Inspect the data for 'malpractice' in 2010

Let's look at the word 'malpractice' in 2010.

In [97]:
word1_2010_speechnumbers

7016     7
7056     6
6966     4
19370    4
26273    4
6968     3
8904     3
29923    3
50935    3
6882     2
Name: speech, dtype: int64

We saw above that the word "malpractice" was invoked far more frequently in 2010 than in 1967.  Here, we see that the speeches that invoked 'malpractice' in 2010 also invoke that term more frequently -- up to 7 times in one speech -- than the one or two times each speech used the term in 1967.  

Such a shift suggests that the word 'malpractice' is being applied with a more specific meaning in 2010. But to interpret this shift, we need to read more.

Let's look at the top speech, from row 7016.  Who gave the speech?.

In [99]:
word1_2010_context['speaker'][7016]

'Mr. SMITH of Texas'

What did the speech say?

In [100]:
word1_2010_context['speech'][7016]

'Madam Speaker. I yield myself such time as I may consume. Madam Speaker. on October 9. the Congressional Budget Office pronounced that a tort reform or civil justice reform package would reduce the Federal budget deficit by an estimated $54 billion over 10 years. CBO recognizes that civil justice reforms also have an impact on the practice of defensive medicine. Defensive medicine is when doctors order more tests or procedures than are necessary just to protect themselves from frivolous lawsuits. Studies show that defensive medicine does not advance patients care or enhance a physicians capabilities. that billions of dollars in savings from tort reform could be used to provide health insurance for the uninsured without raising taxes on those who already have insurance policies. As the administration rushes to enact a massive government takeover of health care. Congress must remember that there is the option of saving between $54 billion and more than $200 billion by embracing tort ref

Here is a sample sentence showing how 'malpractice' is being talked about:

"Medical malpractice premiums have risen more than 80 percent each year in some parts of the country and can cost almost half a million dollars a year in some specialties."

If we read the entire speech, we learn that the debate is about the rising cost of health care.  Many Americans fear that frequent lawsuits are driving up the cost of insurance.

"In the poll. 59 percent said they thought at least half the tests doctors order are unnecessary and ordered only because of fear of lawsuits."

The issue isn't a rise in malpractice, but the apparent fact that doctors' fears of malpractice have been driving up the cost of healthcare.  We learn by reading that the proposed solution was "tort reform" or "medical liability reform," which would have limited the degree to which doctors could be held responsible for malpractice.  

Reading further, we learn that tort reform became an issue for the Democrats, whose plans for universal health insurance promised to make healthcare affordable for all.  

### Perform the same search with new vocabulary

When you research shifts in words, typically you won't work on a single word at a time. You'll begin with a controlled vocabulary of the kind we used to investigate discussions of crime in Congress last week. Then you'll narrow down to a few words whose meaning you need to understand better.

We can use a 'for' loop to cycle through a dozen words very quickly, producing an analysis.  

We can even add a few lines to this for loop to ask the computer to print out all the relevant speeches.  I have abbreviated the number of words listed and the lines of the speeech given for room.

When you do your assignment, begin by zeroing in on certain words in the list of keywords on the first line.  

Next, tweak the code to show more words and lines of the speech when you get to your own research.



In [113]:
for word1 in ['rape', 'slaughter', 'torture', 'felony', 'robbery', 'shell game', 'juvenile delinquency', 
             'gambling', 'massacre', 'homicide', 'scam', 'malpractice']:
    
  # get the rownumbers of the n speeches that use word1 the most
    word1_1967_speechnumbers = congress1967['speech'].str.count(word1).nlargest(2) 
    word1_2010_speechnumbers = congress2010['speech'].str.count(word1).nlargest(2) 

    # get the speeches that mentioned word1 the most
    word1_1967_context = congress1967.loc[list(word1_1967_speechnumbers.index)]
    word1_2010_context = congress2010.loc[list(word1_2010_speechnumbers.index)]

    # clean up the data
    word1_1967_context['speech'] = word1_1967_context['speech'].str.replace('[^\w\s]','').str.lower() # remove punctuation, lowercase
    word1_1967_context['stopworded'] = word1_1967_context['speech'].str.replace(stopwords_regex, '') #stopwording

    word1_2010_context['speech'] = word1_2010_context['speech'].str.replace('[^\w\s]','').str.lower() # remove punctuation, lowercase
    word1_2010_context['stopworded'] = word1_2010_context['speech'].str.replace(stopwords_regex, '') #stopwording

    # print some context for 1967
    print("*****")
    print("CONTEXT for "+word1+" in 1967")
    for speechnumber in list(word1_1967_speechnumbers.index):
        speaker = congress1967['speaker'][speechnumber]
        date = congress1967['date'][speechnumber]
        print('speech by '+ speaker+ ' in ' + date)
        print(congress1967['speech'][speechnumber][:100])

    # count the other words that appear in the context of word1 in 1967
    word1_1967_context_count = word1_1967_context["stopworded"].str.split().explode().dropna().value_counts()
    print("***")
    print("COLLOCATES for " + word1 + " in 1967")
    print(word1_1967_context_count[:5])

    # print some context for 2010
    print("*****")
    print("CONTEXT for "+word1+" in 2010")
    for speechnumber in list(word1_2010_speechnumbers.index):
        speaker = congress2010['speaker'][speechnumber]
        date = congress2010['date'][speechnumber]
        print('speech by '+ speaker+ ' in ' + date)
        print(congress2010['speech'][speechnumber][:100])

    # count the other words that appear in the context of word1 in 2010
    word1_2010_context_count = word1_2010_context["stopworded"].str.split().explode().dropna().value_counts()
    print("COLLOCATES for " + word1 + " in 2010")
    print(word1_2010_context_count[:5])

*****
CONTEXT for rape in 1967
speech by Mr. ERVIN in 1967-01-23
Mr. President. in its recent 5 to 4 decision. in Miranda v. Arizona. 384 U.S. 436. the Supreme Court
speech by Mrs. BOLTON in 1967-07-17
Mr. Speaker. the city of South Euclid. Ohio. in the 22d Congressional District of Ohio. is celebrati
***
COLLOCATES for rape in 1967
court      33
case       28
law        23
miranda    21
states     21
Name: stopworded, dtype: int64
*****
CONTEXT for rape in 2010
speech by Mr. SHADEGG in 2010-04-27
Mr. Chairman. I rise in support of the en bloc amendment. but in particular in support of my amendme
speech by Mr. LEAHY in 2010-09-27
Mr. President. today. I am proud to introduce the Justice for All Reauthorization Act of 2010. toget
COLLOCATES for rape in 2010
justice    23
rape       18
act        13
law        13
crime      10
Name: stopworded, dtype: int64
*****
CONTEXT for slaughter in 1967
speech by Mr. SMITH of Iowa in 1967-02-27
Mr. Speaker. I have been introducing and promoting bil

## Assignment

You will write an essay of approximately one page that explains how both keywords changed their meaning.

Use the code above to compare the context for TWO keywords of your choosing about crime. Tweak the code to show at least 30 collocates and the entire speech for each line of context.  

Choose words whose shift in context you believe you mostly understand, but about which you can discover something new. 

Investigate through the following steps.  

* Compare how frequently the keywords were talked about in each time period. 
* Use the collocates chart to compare how regularly the keyword’s collocates were invoked with that keyword in each year. Use the bar chart of collocates in your answer.  
* Make an argument about how each keyword’s usage changed from 1967 to 2010.  Did the keyword go from being unimportant to important? Did specific cases mean that the keyword was being invoked in a new and specific sense? Or did a keyword that was once important disappear in significance?
* Use short quotations of at least three words and no more than one line to support your answer. Use at least ONE sentences of context from speeches where the word appears frequently to support each keyword-era combination. Cite that context using turabian style in a footnote, e.g., SpeakerName, Speech, U.S. House of Representatives (Date).

***Important***: It is possible that you might go through these steps and still not understand how a word changed.  If this happens (and it will), you have two options: 
1) keep digging. look at more collocates and read more context.  
2) decide that the shift in question is too nuanced for you.  choose a different keyword to research.

Only write about a keyword if you believe you have successfully reached understanding.